In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

def clean_player_name(name):
    name = name.replace("Williams S.", "Williamsx S.")
    name = name.replace("Radwanska U.", "Radwanskax U.")
    name = name.replace("Pliskova Ka.", "Pliskovax K.")
    name = name.replace("Pliskova Kr.", "Pliskova K.")
    name = name.replace("Rodionova Ar.", "Rodionova A.")
    name = name.replace("Rodionova An.", "Rodionovax A.")
    return name

shared_links = [
    "https://drive.google.com/file/d/1jKMWbeXCFWnEFsOva5tzFMUSt5f8k2qg/view?usp=drive_link",
    "https://drive.google.com/file/d/1WMY_ntO2_MU4qnVRc5UaebcUEnv6Hvvb/view?usp=drive_link",
    "https://drive.google.com/file/d/1bkWAkYqX9rbsGX_gt7A5ApXMYzXBh0Wq/view?usp=drive_link",
    "https://drive.google.com/file/d/1bkQ88Ph0jLGnUvil6fvlx9KxOhfFjA2l/view?usp=drive_link",
    "https://drive.google.com/file/d/1KkejTH65MOesD7KRitUKV5MMmCOAvmfS/view?usp=drive_link",
    "https://drive.google.com/file/d/1Svt9MUuAeVFBlz3hiwAJL6-yra_PUg20/view?usp=drive_link",
    "https://drive.google.com/file/d/1eHvMgpONaD9mS-SSeFvclKXDpLkqXGHO/view?usp=drive_link",
    "https://drive.google.com/file/d/1Z9-vGHjp4t1USzVvCP7Lz7_PnEpGRtuf/view?usp=drive_link",
    "https://drive.google.com/file/d/1p9Y3fFagDkpNTRkVrYCxEyrpZi6-K3pv/view?usp=drive_link"
]

dfs = []
for year, link in zip(range(2015, 2024), shared_links):
    direct_link = get_direct_link(link)
    df = pd.read_csv(direct_link, encoding='utf-8')
    df["Year"] = year
    
    df['Winner'] = df['Winner'].apply(clean_player_name)
    df['Loser'] = df['Loser'].apply(clean_player_name)
    
    dfs.append(df)

combined_df = pd.concat(dfs)

combined_df['LastName_W'] = combined_df['Winner'].str.extract(r'(.*)\s\w\.$')
combined_df['LastName_L'] = combined_df['Loser'].str.extract(r'(.*)\s\w\.$')

combined_df['LastName_W'] = combined_df['LastName_W'].str.strip()
combined_df['LastName_L'] = combined_df['LastName_L'].str.strip()

In [3]:
combined_df.reset_index(inplace=True)

In [4]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%d/%m/%Y')

start_date = pd.to_datetime('01/06/2015', format='%d/%m/%Y')
end_date = pd.to_datetime('30/07/2023', format='%d/%m/%Y')

trimmed_df = combined_df.loc[(combined_df['Date'] >= start_date) & (combined_df['Date'] <= end_date)]

In [5]:
trimmed_df.reset_index(inplace=True)

In [6]:
combined_df = trimmed_df[['Location', 'Tournament', 'Date', 'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'B365W', 'B365L', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'Year']]

In [7]:
combined_df.shape

(18817, 15)

In [8]:
combined_df.to_csv('unclean_odds.csv', index=False)

In [9]:
disagree_rows = combined_df[((combined_df['B365W'] < combined_df['B365L']) & 
                            (combined_df['AvgW'] > combined_df['AvgL'])) | 
                           ((combined_df['B365W'] > combined_df['B365L']) & 
                            (combined_df['AvgW'] < combined_df['AvgL']))]
significant_disagree_rows = disagree_rows[abs(disagree_rows['B365W'] - disagree_rows['B365L']) > 0.2]
significant_disagree_rows

,Location,Tournament,Date,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year
564,Toronto,Rogers Cup,2015-08-11,1st Round,Errani S.,Mladenovic K.,17.0,39.0,1.72,2.00,2.01,2.00,1.90,1.86,2015
3101,Florianopolis,Brasil Tennis Cup,2016-08-05,The Final,Begu I.,Babos T.,31.0,44.0,2.37,1.57,2.65,2.15,1.82,1.98,2016
6034,Wuhan,Wuhan Open,2017-09-25,1st Round,Goerges J.,Gavrilova D.,27.0,22.0,2.00,1.72,2.03,2.15,1.85,1.95,2017
7692,Eastbourne,Eastbourne International,2018-06-25,1st Round,Hsieh S.W.,Vikhlyantseva N.,52.0,106.0,2.00,1.72,2.00,2.04,1.90,1.91,2018
9416,Miami,Sony Ericsson Open,2019-03-23,2nd Round,Hercog P.,Doi M.,93.0,112.0,5.50,1.14,2.10,2.37,1.85,2.02,2019
11941,Istanbul,Istanbul Cup,2020-09-12,Semifinals,Bouchard E.,Badosa P.,272.0,94.0,2.75,1.44,2.93,2.32,1.81,2.07,2020
12003,New York,US Open,2020-09-02,1st Round,Pera B.,Diyas Z.,61.0,69.0,1.80,1.50,2.38,2.61,1.58,2.41,2020
13124,Charleston,MUSC Health Women's Open,2021-04-16,Quarterfinals,Sharma A.,Fruhvirtova L.,165.0,499.0,1.72,2.10,2.54,2.10,2.11,1.74,2021
13638,Bad Homburg,Bad Homburg Open,2021-06-21,1st Round,Siegemund L.,Bhatia R.,55.0,362.0,3.00,1.40,1.50,2.61,1.50,2.61,2021
14852,Adelaide,Adelaide International 1,2022-01-10,1st Round,Kostyuk M.,Rogers S.,50.0,36.0,2.62,1.50,2.90,1.53,1.47,2.64,2022


In [10]:
def swap_odds(row_index, bookmakers, dataframe):
    for bookmaker in bookmakers:
        w_column = f"{bookmaker}W"
        l_column = f"{bookmaker}L"
        
        dataframe.at[row_index, w_column], dataframe.at[row_index, l_column] = dataframe.at[row_index, l_column], dataframe.at[row_index, w_column]
    
    return dataframe

In [11]:
combined_df = swap_odds(13124, ['B365'], combined_df)
combined_df = swap_odds(14852, ['Avg'], combined_df)
combined_df = swap_odds(18672, ['B365'], combined_df)

In [12]:
disagree_rows = combined_df[((combined_df['B365W'] < combined_df['B365L']) & 
                            (combined_df['AvgW'] > combined_df['AvgL'])) | 
                           ((combined_df['B365W'] > combined_df['B365L']) & 
                            (combined_df['AvgW'] < combined_df['AvgL']))]
significant_disagree_rows = disagree_rows[abs(disagree_rows['B365W'] - disagree_rows['B365L']) > 0.2]
significant_disagree_rows

,Location,Tournament,Date,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year
564,Toronto,Rogers Cup,2015-08-11,1st Round,Errani S.,Mladenovic K.,17.0,39.0,1.72,2.00,2.01,2.00,1.90,1.86,2015
3101,Florianopolis,Brasil Tennis Cup,2016-08-05,The Final,Begu I.,Babos T.,31.0,44.0,2.37,1.57,2.65,2.15,1.82,1.98,2016
6034,Wuhan,Wuhan Open,2017-09-25,1st Round,Goerges J.,Gavrilova D.,27.0,22.0,2.00,1.72,2.03,2.15,1.85,1.95,2017
7692,Eastbourne,Eastbourne International,2018-06-25,1st Round,Hsieh S.W.,Vikhlyantseva N.,52.0,106.0,2.00,1.72,2.00,2.04,1.90,1.91,2018
9416,Miami,Sony Ericsson Open,2019-03-23,2nd Round,Hercog P.,Doi M.,93.0,112.0,5.50,1.14,2.10,2.37,1.85,2.02,2019
11941,Istanbul,Istanbul Cup,2020-09-12,Semifinals,Bouchard E.,Badosa P.,272.0,94.0,2.75,1.44,2.93,2.32,1.81,2.07,2020
12003,New York,US Open,2020-09-02,1st Round,Pera B.,Diyas Z.,61.0,69.0,1.80,1.50,2.38,2.61,1.58,2.41,2020
13638,Bad Homburg,Bad Homburg Open,2021-06-21,1st Round,Siegemund L.,Bhatia R.,55.0,362.0,3.00,1.40,1.50,2.61,1.50,2.61,2021
18414,Berlin,German Open,2023-06-20,1st Round,Avanesyan E.,Kasatkina D.,81.0,11.0,1.04,13.00,2.91,1.41,2.91,1.41,2023
18518,Eastbourne,Eastbourne International,2023-06-27,1st Round,Wang Xiy.,Marino R.,73.0,83.0,2.50,1.53,1.80,2.05,1.79,2.02,2023


In [13]:
idx_drop = [3101, 9416, 11941, 12003, 13638, 18414, 18518]
combined_df = combined_df.drop(idx_drop)

In [14]:
def filter_invalid_probabilities(df, bookmakers):
    odds1_col = f'{bookmakers}W'
    odds2_col = f'{bookmakers}L'
    
    df['prob1'] = 1 / df[odds1_col]
    df['prob2'] = 1 / df[odds2_col]

    invalid_rows = df[(df['prob1'] + df['prob2'] < 1) | (df['prob1'] + df['prob2'] > 1.1)]

    return invalid_rows

In [15]:
invalid_df = filter_invalid_probabilities(combined_df, 'Avg')
invalid_df

,Location,Tournament,Date,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year,prob1,prob2
2638,Birmingham,AEGON Classic,2016-06-15,1st Round,Gavrilova D.,Broady N.,50.0,84.0,1.4,2.75,1.52,3.00,1.42,2.18,2016,0.704225,0.458716
13735,London,Wimbledon,2021-06-30,1st Round,Putintseva Y.,Pironkova T.,43.0,103.0,NaN,NaN,1.78,15.75,1.40,8.90,2021,0.714286,0.112360


In [16]:
invalid_df = filter_invalid_probabilities(combined_df, 'B365')
invalid_df

,Location,Tournament,Date,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year,prob1,prob2
2435,Strasbourg,Internationaux de Strasbourg,2016-05-19,Quarterfinals,Mladenovic K.,Kudryavtseva A.,29.0,184.0,1.3,3.00,1.37,3.65,1.31,3.34,2016,0.769231,0.333333
6765,Acapulco,Abierto Mexicano,2018-02-26,1st Round,Cepede Royg V.,Bencic B.,82.0,70.0,4.0,1.11,6.39,1.29,4.39,1.17,2018,0.250000,0.900901
14048,Montreal,Canadian Open,2021-08-10,1st Round,Vekic D.,Van Uytvanck A.,56.0,60.0,1.8,2.50,1.84,2.20,1.73,2.09,2021,0.555556,0.400000


In [17]:
idx_drop = [13735,6765,14048]
combined_df = combined_df.drop(idx_drop)

In [18]:
combined_df.reset_index(inplace=True)

In [19]:
combined_df['AvgW_prob'] = 1 / combined_df['AvgW']
combined_df['AvgL_prob'] = 1 / combined_df['AvgL']
combined_df['B365W_prob'] = 1 / combined_df['B365W']
combined_df['B365L_prob'] = 1 / combined_df['B365L']

combined_df['W_diff'] = abs(combined_df['AvgW_prob'] - combined_df['B365W_prob'])
combined_df['L_diff'] = abs(combined_df['AvgL_prob'] - combined_df['B365L_prob'])

In [20]:
Q1 = combined_df['W_diff'].quantile(0.25)
Q3 = combined_df['W_diff'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = combined_df[(combined_df['W_diff'] >= lower_bound) & (combined_df['W_diff'] <= upper_bound)]
outliers_removed = combined_df.shape[0] - filtered_data.shape[0]
print(outliers_removed)
combined_df = filtered_data

575


In [21]:
Q1 = combined_df['L_diff'].quantile(0.25)
Q3 = combined_df['L_diff'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = combined_df[(combined_df['L_diff'] >= lower_bound) & (combined_df['L_diff'] <= upper_bound)]
outliers_removed = combined_df.shape[0] - filtered_data.shape[0]
print(outliers_removed)
combined_df = filtered_data

455


In [22]:
columns_choice = ['Date', 'Year', 'Winner', 'Loser', 'Round', 'B365W', 'B365L', 'AvgW', 'AvgL', 'MaxW', 'MaxL', 'WRank', 'LRank']
columns_to_drop = [col for col in combined_df.columns if col not in columns_choice]
combined_df = combined_df.drop(columns=columns_to_drop)
combined_df.reset_index(inplace=True)

In [23]:
combined_df.shape

(17777, 14)

In [24]:
combined_df.to_csv('odds.csv', index=False)